##### For English, please click <a href="#header-en">here</a> to redirect

<h1 id="header-ch">螺旋桨RNA结构预测竞赛第六名方案</h1>

**任务**：对于给定的RNA碱基序列，要求构建模型预测RNA碱基不成对的概率。

**工程环境**：Python3.7+PaddlePaddle2.0.2，fork官方基线系统；

Baseline方案：lstm

# 赛题介绍

“RNA碱基不成对概率”衡量了RNA序列在各个点位是否能形成稳定的碱基对（base pair），是RNA结构的重要属性，并可被应用在mRNA疫苗序列设计、药物研发等领域。例如mRNA疫苗序列通常不稳定，而RNA碱基不成对概率较高的点位正是易被降解的位置；又如RNA 碱基不成对概率较高的点位通常更容易与其他RNA序列相互作用，形成RNA-RNA binding等，这一特性也被广泛应用于疾病诊断和RNA药物研发。

本次比赛提供了5000条训练数据，选手需要基于训练数据和飞桨平台，开发模型预测RNA碱基不成对概率。

（<span style='color:red'>Tips：机器学习框架方面只允许使用飞桨深度学习框架哦</span>）

# 竞赛数据集 

初赛数据集包含训练集4750条，验证集250条，测试集444条，采用官方给定的数据进行训练，不再单独划分；

In [6]:
# 检查数据集所在路径，数据放置路径
# 其中train.txt/dev.txt/test_nolabel.txt分别为训练集/验证集/A榜测试集，B_board_112_seqs.txt为B榜测试集，train_dev2.txt为训练集合并了2遍验证集生成的文件
!tree /home/aistudio/work/data

## 方案分析及模型训练

`python src/main.py train --model-path-base [model_directory_name]`

本代码会训练一个模型，并且保存到指定位置，训练日志默认保存到文件`train_log.txt`   
注意：由于初始化的不稳定，可能需要多次训练，比较合理的验证集(dev)均方误差损失值(MSE loss)为0.05-0.08  

#### 样例
`python src/main.py train --model-path-base model`

#### 你将会看到类似如下的训练日志
```
epoch 1 batch 40 processed 640 batch-loss 0.1984 epoch-elapsed 0h00m10s total-elapsed 0h00m11s 
epoch 1 batch 41 processed 656 batch-loss 0.2119 epoch-elapsed 0h00m10s total-elapsed 0h00m11s 
epoch 1 batch 42 processed 672 batch-loss 0.2205 epoch-elapsed 0h00m11s total-elapsed 0h00m11s 
epoch 1 batch 43 processed 688 batch-loss 0.2128 epoch-elapsed 0h00m11s total-elapsed 0h00m11s 
# Dev Average Loss: 0.212 (MSE) -> 0.461 (RMSD)
```

#### 注意事项
请使用<span style='color:red'>GPU版本</span>的配置环境运行本模块




**直接无休改训练baseline网络：**

本方案最终生成的模型位置：work/model/model_bl_self=0.0676

In [ ]:
# To train:
# python src/main.py train --model-path-base [model_directory_name]

!cd work; python src/main.py train --model-path-base model

**修改激活函数**

将网络自带的Relu激活函数修改为swish激活函数，该工程为work2;

修改代码：work2/src/network.py

模型位置：work2/model/model_dev=0.0700


In [ ]:
!cd work2; python src/main.py train --model-path-base model

**全数据训练**

充足的训练数据才是模型效果的保障，因此最后考虑采用训练加验证合并训练的方法，同时为了使新增数据得到充分训练，将其扩充为2倍加入到训练集中，训练baseline网络；

***注意：需在work/src/dataset.py中第33和35行，将train.txt修改为traindev2.txt***

数据文件：work/data/train_dev2.txt

模型位置：work/model/model_add_dev=0.0624


In [ ]:
!cd work; python src/main.py train --model-path-base model

**增大网络训练**

将baseline网络增大一倍，layers参数由8设置为16，dmodel参数由128修改设置为256；

模型位置：work/model_x2/model_x2_dev=0.0660

***注意：由于结果不理想，本方案最终并未使用此模型***

In [ ]:
!cd work; python src/main.py train --model-path-base model --dmodel 256 --layers 16

**增加训练epoch**

将训练epoch，由10增加为20，训练baseline网络，效果不理想，模型最终未保存；

***注意：由于结果不理想，本方案最终并未使用此模型***

In [ ]:
!cd work; python src/main.py train --model-path-base model  --epochs 20

## 模型预测

`python src/main.py test --model-path-base [saved_model_directory]`  

本代码会预测一个模型，日志和结果默认保存到文件`test_log.txt` 

#### 样例  
1. 用<span style='color:red'>不带标签</span>的测试集来预测：  
`python src/main.py test --model-path-base model-0/model_dev\=0.0772/`
2. 用<span style='color:red'>带标签</span>的测试集来预测并评估:  
	`python src/main.py test_withlabel --model-path-base model-0/model_dev\=0.0772/`  
	样例输出
    ```
    # python3 src/main.py test_withlabel --model-path-base model-0/model_dev=0.0772
    Loading data...
    Loading model...
    initializing vacabularies... done.
    Sequence(6): ['<START>', '<STOP>', 'A', 'C', 'G', 'U']
    Brackets(5): ['<START>', '<STOP>', '(', ')', '.']
    W0113 21:57:44.871776   221 device_context.cc:252] Please NOTE: device: 0, CUDA Capability: 70, Driver API Version: 11.0, Runtime API Version: 9.0
    W0113 21:57:44.878015   221 device_context.cc:260] device: 0, cuDNN Version: 7.6.
    #  Dev Average Loss: 0.0772 (MSE) -> 0.2778 (RMSD)
    # Test Average Loss: 0.0445 (MSE) -> 0.2111 (RMSD)
    ```

- 由于比赛的公开数据不提供测试集的标签，故本基线模型无法运行预设的`test_withlabel`，除非用户自己生成一个带标签的测试集`~/data/test.txt`。  




#### 注意事项
请使用<span style='color:red'>GPU版本</span>的配置环境运行本模块

**预测baseline模型的结果**



In [ ]:
# To test 1:
# python src/main.py test --model-path-base [saved_model_directory]
!cd work; python src/main.py test --model-path-base model-0/model_dev\=0.0772
# 执行完成后会在work目录生成test_log.txt，然后执行：
!cd work; python create_result.py
# 生成predict.files文件夹即为可提交的结果，并重命名为predict.files_bl；
!cd work; mv predict.files predict.files_bl

**预测无修改训练的baselline网络模型结果**

In [ ]:
!cd work; python src/main.py test --model-path-base model/ model_bl_self=0.0676
# 执行完成后会在work目录生成test_log.txt，然后执行：
!cd work; python create_result.py
# 生成predict.files文件夹即为可提交的结果，并重命名为predict.files_bl_self；
!cd work; mv predict.files predict.files_bl_self

**预测修改激活函数的模型结果**

In [ ]:
!cd work2; python src/main.py test --model-path-base model/ model_bl_self=0.0700
# 执行完成后会在work目录生成test_log.txt，然后执行：
!cd work2; python create_result.py
# 生成predict.files文件夹即为可提交的结果，并重命名为predict.files_bl_self；
!cd work2; mv predict.files predict.files_bl_swish

**预测全数据集训练的模型**

In [ ]:
!cd work; python src/main.py test --model-path-base model/ model_bl_self=0.0624
# 执行完成后会在work目录生成test_log.txt，然后执行：
!cd work; python create_result.py
# 生成predict.files文件夹即为可提交的结果，并重命名为predict.files_bl_self；
!cd work; mv predict.files predict.files_bl_add_dev

In [ ]:
# To test 2:
# python src/main.py test_withlabel --model-path-base [saved_model_directory]

# 由于比赛的公开数据不提供测试集的标签，故本基线模型无法运行预设的test_withlabel，除非用户自己生成一个带标签的测试集~/data/test.txt
#### !cd work; python src/main.py test_withlabel --model-path-base model-0/model_dev\=0.0772

# 模型融合

在上述方案中，如原baseline网络训练多次，很多效果不是很理想，仅保留了其中最好的结果的模型。然后采用多个模型结果融合的方法进一步提升指标。
模型融合采用4个模型，分别是：
1. baseline模型
2. 无修改训练的baseline网络模型
3. 修改激活函数的模型
4. 全数据训练模型

融合方式采用线性加权的方式，为4个模型生成的概率值分别赋给不同的权重然后生成融合后的结果；

初赛中，通过分配不同的比例的权重进行多次的对比与测试，最终采用的融合比例为0.8/0.1/0.05/0.05，初赛为第20名；

最终的结果：postprocess/predict.files.zip

复赛中，由于不同模型对复赛的测试集的效果与初赛有很大的不同，因此在分别测试了单个模型的结果后，需要对比例进行调整和验证，最终的比例为0.45/0.01/0/0.45，复赛为第6名；

最终的结果：postprocess_2/predict.files.zip


In [ ]:
#　将模型预测的４个文件夹都拷贝到postprocess_2文件夹中；
!cd postprocess_2; cp -r work/predict.files_*  ./
!cd postprocess_2; cp -r work2/predict.files_*  ./
# 然后执行postprocess.py，融合结果
!cd postprocess_2; python postprocess.py
# 执行后生成的predict.files文件夹，就是最终的结果，再压缩就可以提交；
!cd postprocess_2; zip -r predict.files.zip predict.files

***说明：***

***1. 以上步骤为生成复赛结果的步骤；***
    
***2. 如果需要生成初赛的结果，需要修改work/src/dataset.py中line40和line41的测试集路径，将Bboard112seqs.txt修改为testnolabel.txt，work2同理；***
    
***3. 然后按照模型预测的步骤生成4个文件夹，然后拷贝到postprocess文件夹中，检查并修改postprocess.py中的这四个文件夹的文件名，然后执行该文件即可；***